In [ ]:
import torch
from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

In [ ]:
batch

In [ ]:
batch.keys()

In [ ]:
batch["labels"]= torch.tensor([1,1])

In [ ]:
batch

In [ ]:
output = model(**batch)

In [ ]:
loss = 0.2

In [ ]:
optimizer = AdamW(model.parameters())

In [ ]:
loss = model(**batch)*

In [ ]:
loss.backward()

In [ ]:
optimizer.step

In [ ]:
# MRPC dataset! This is one of the 
# 10 datasets composing the GLUE benchmark, which is an academic 
# benchmark that is used to measure the performance of ML models across 
# 10 different text classification tasks.

In [ ]:
from datasets import load_dataset

In [ ]:
# This command downloads and caches the 
# dataset, by default in ~/.cache/huggingface/datasets.
# Recall from Chapter 2 that you can customize your cache 
# folder by setting the HF_HOME environment variable.



In [ ]:
# BERT is pretrained with token type IDs, and on top 
# of the masked language modeling objective we talked about 
# in Chapter 1, it has an additional objective called next 
# sentence prediction. The goal with this task is 
# to model the relationship between pairs of sentences.

In [ ]:
#With next sentence prediction, the model is provided pairs of 
# sentences (with randomly masked tokens) and asked to predict
#  whether the second sentence follows the first. To make the 
# task non-trivial, half of the time the sentences follow each 
# other in the original document they were extracted from, and the other half 
# of the time the two sentences come from two different documents

In [ ]:
from datasets import load_dataset 

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")

In [ ]:
raw_datasets.keys()

In [ ]:
raw_datasets['train']

In [ ]:
raw_datasets['train'].features['sentence1']

In [ ]:
raw_datasets['train'].features

In [ ]:
raw_datasets['train'].features['sentence1']

In [ ]:
raw_datasets

In [ ]:
raw_datasets['train'][0]

In [ ]:
from transformers import AutoTokenizer
checkpoint = "bert-base-uncased"
tk = AutoTokenizer.from_pretrained(checkpoint)


In [ ]:
raw_datasets['train'][0]['sentence1']

In [ ]:
raw_token_test = tk(raw_datasets['train'][0]['sentence1'], raw_datasets['train'][0]['sentence2'])

In [ ]:
raw_token_test

In [ ]:
# this takes two sentences as one. 
# token id type is used to recoginzie which senten

In [ ]:
raw_datasets['train']['sentence1'][0:10]

In [ ]:
raw_token_test = tk(raw_datasets['train']['sentence1'][0:10], raw_datasets['train']['sentence2'][0:10])

In [ ]:
raw_token_test

In [ ]:
raw_token_test['input_ids']

In [ ]:
tk.convert_ids_to_tokens(raw_token_test["input_ids"][1])

In [ ]:
tokenized_dataset = tk(raw_datasets['train']['sentence1'], raw_datasets['train']['sentence2'], padding=True,truncation=True)

In [ ]:
tokenized_dataset.keys()

In [ ]:
# this works, you can use the whole tokenzied dataset in your memoory but this is not efficient.

In [ ]:
def tokenize_function(example):
    # generally you should simply pass a string, string kinda thing 
    # doesn't matter if its train or test or anything
    # Note that we’ve left the padding argument out in 
    # our tokenization function for now. This is 
    # because padding all the samples to the maximum 
    # length is not efficient: it’s better to pad the 
    # samples when we’re building a batch, as then we 
    # only need to pad to the maximum length in that batch, 
    # and not the maximum length in the entire dataset. T
    return tk(example['sentence1'], example['sentence2'], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# Here is how we apply the tokenization function on all our datasets at once.
#You can even use multiprocessing when 
# applying your preprocessing function with map() 
# by passing along a num_proc argument. We didn’t 
# do this here because the 🤗 Tokenizers library already 
# uses multiple threads to tokenize our samples faster, but 
# if you are not using a fast tokenizer backed by this library, 
# this could speed up your preprocessin

In [ ]:
tokenized_datasets

In [ ]:
#The last thing we will need to do is pad all the examples to the length of the longest element when we batch elements 
# together — a technique we refer to as dynamic padding.

In [ ]:
tokenized_datasets.keys()

In [ ]:
tokenized_datasets['train'][0]['input_ids']

In [ ]:
# so we have created tokens from all parts of dataset using 
# raw_data_set.map function with batch.
# no padding was used as we want to pad batch by batch at least.
#This will speed up training by quite a bit, but note that if you’re training on a TPU it can cause problems — 
# TPUs prefer fixed shapes, even when that requires extra padding.


In [ ]:
# For applying padding to each batch based seperately,
# we should do dynamic padding
#The function that is responsible for putting 
# together samples inside a batch is called a collate function.

In [ ]:
# To do this in practice, we have to define a collate function that 
# will apply the correct amount of padding to the 
# items of the dataset we want to batch together


In [ ]:
# Transformers library provides us with such a function via DataCollatorWithPadding.

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(tk)

In [ ]:
data_collator = DataCollatorWithPadding(tk)


In [ ]:
data_collator

In [ ]:
# lets take some samples from our tokenized data set we batched together
samples = tokenized_datasets['train'][0:10]

In [ ]:
samples.keys()

In [ ]:
print(type(tokenized_datasets['train']))

In [ ]:
tokenized_datasets['train']

In [ ]:
# This is a data set object. 

In [ ]:
print(type(tokenized_datasets['train'][:8]))

In [ ]:
tokenized_datasets['train'][:8].keys()

In [ ]:
samples = tokenized_datasets['train'][:8]

In [ ]:
samples['sentence1']

In [ ]:
samples['sentence2']

In [ ]:
# the goal is to only keep everything execept sentence 1 and sentence 2 and idx
# as we will pass the toke ids with token type id and attetion mask for this batch 
# thrroigh tensor
# so we just need sub-dictionary
# if we have sub-dictionary which doesn't have  sentence 1 and sentence 2 and idx, that should work


In [ ]:
samples = {k:v for k, v in samples.items() if k not in ["sentence1", "sentence2", "idx"]}

In [ ]:
samples.keys()

In [ ]:
samples["input_ids"]

In [ ]:
[len(v) for v in samples["input_ids"]]

In [ ]:
# we see length of input ids is different. we need to make it same by padding
# dynamic padding should help out

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tk)

In [ ]:
batch = data_collator(samples)

In [ ]:
batch.keys()

In [ ]:
[len(x) for x in batch['input_ids']]

In [ ]:
# with DataCollatorWithPadding we got padding applied to the specifuc batch.
# DataCollatorWithPadding must take tokenizer as input as it should know how to 
# apply tokenizr

In [ ]:
batch["input_ids"].shape

In [ ]:
# DataCollatorWithPadding also create tensors our of the ids.

In [ ]:
# lets check the size of each key in the batch

In [ ]:
{k:v.shape for k, v in batch.items()}

In [ ]:
# DataCollatorWithPadding converts input ids, token type ids, attention mask and label to tensor
# of the same length in a batch


In [ ]:
[v.shape for k, v in batch.items()]